# 1 - Packages

In [ ]:
import numpy as np
import pickle as pkl
import tensorflow as tf

from tensorflow import keras

# 2 - Constants

In [ ]:
CONSTANTS_PATH = '../../../helpers/constants'
DATASET_PATH = '../../../dataset'
CHARS_NUM = 50
CLASSES_NUM = 15

with open(CONSTANTS_PATH + '/ARABIC_LETTERS_LIST.pickle', 'rb') as file:
    ARABIC_LETTERS_LIST = pkl.load(file)

with open(CONSTANTS_PATH + '/DIACRITICS_LIST.pickle', 'rb') as file:
    DIACRITICS_LIST = pkl.load(file)

with open(CONSTANTS_PATH + '/FFNN_SMALL_CHARACTERS_MAPPING.pickle', 'rb') as file:
    CHARACTERS_MAPPING = pkl.load(file)

with open(CONSTANTS_PATH + '/FFNN_CLASSES_MAPPING.pickle', 'rb') as file:
    CLASSES_MAPPING = pkl.load(file)

with open(CONSTANTS_PATH + '/FFNN_REV_CLASSES_MAPPING.pickle', 'rb') as file:
    REV_CLASSES_MAPPING = pkl.load(file)

# 3 - Load the Model

In [ ]:
model = keras.models.load_model('model.ckpt')
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.train.AdamOptimizer(),
              metrics=['accuracy'])
model.summary()

# 4 - Evaluate the Model

In [ ]:
X_test = None
Y_test = None
with open(DATASET_PATH + '/X_test.pickle', 'rb') as X_test_file, \
         open(DATASET_PATH + '/Y_test.pickle', 'rb') as Y_test_file:
    X_test = pkl.load(X_test_file)
    Y_test = pkl.load(Y_test_file)
print('Testing examples:', len(X_test))

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, Y, batch_size):
        self.X, self.Y = X, Y
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.X) / float(self.batch_size)))

    def __getitem__(self, idx):
        X_batch = np.asarray(self.X[idx * self.batch_size:(idx + 1) * self.batch_size])
        Y_batch = np.asarray(self.Y[idx * self.batch_size:(idx + 1) * self.batch_size])
        
        X_tmp = list()
        for x in X_batch:
            before_need = x[0]
            after_need = x[-1]
            x_new = list()
            x_new.extend([1] * before_need)
            x_new.extend(x[1:-1])
            x_new.extend([1] * after_need)
            X_tmp.append(np.asarray(x_new))
        
        X_batch = np.asarray(X_tmp)
        Y_batch = np.asarray(Y_batch)
        
        return X_batch, Y_batch

In [ ]:
test_generator = DataGenerator(X_test, Y_test, len(X_test))

XY_test = list()
for batch in range(len(test_generator)):
    XY_test.append(test_generator[batch])
X, Y = zip(*XY_test)

X = np.asarray(X)
Y = np.asarray(Y)

X = np.squeeze(X)
Y = np.squeeze(Y)

In [ ]:
loss, acc = model.evaluate(X, Y, batch_size=512)
print('Accuracy: %s%%' % round(acc * 100, 2))
print('Loss: %s' % round(loss, 2))

# 5 - Predicting

In [ ]:
def predict(line):
    equal = 0
    not_equal = 0
    output = ''
    
    for idx, ch in enumerate(line):
        if ch in DIACRITICS_LIST:
            continue
        
        output += ch
        
        if ch not in ARABIC_LETTERS_LIST:
            continue
            
        y_true = [0] * CLASSES_NUM
        if idx + 1 < len(line) and line[idx + 1] in DIACRITICS_LIST:
            ch_diac = line[idx + 1]
            if idx + 2 < len(line) and line[idx + 2] in DIACRITICS_LIST and ch_diac + line[idx + 2] in CLASSES_MAPPING:
                ch_diac += line[idx + 2]
            y_true[CLASSES_MAPPING[ch_diac]] = 1
        else:
            y_true[0] = 1
        y_true = np.asarray(y_true).reshape(1, -1)

        before = list()
        after = list()

        for idxb in range(idx - 1, -1, -1):
            if len(before) >= CHARS_NUM:
                break
            if line[idxb] not in DIACRITICS_LIST:
                before.append(line[idxb])
        before = before[::-1]
        before_need = CHARS_NUM - len(before)

        for idxa in range(idx, len(line)):
            if len(after) >= CHARS_NUM:
                break
            if line[idxa] not in DIACRITICS_LIST:
                after.append(line[idxa])
        after_need = CHARS_NUM - len(after)

        x = list()
        x.extend([1] * before_need)
        for ch in before:
            try:
                x.append(CHARACTERS_MAPPING[ch])
            except:
                x.append(0)
        for ch in after:
            try:
                x.append(CHARACTERS_MAPPING[ch])
            except:
                x.append(0)
        x.extend([1] * after_need)
        x = np.asarray(x)

        x = np.asarray(x).reshape(1, -1)
        y_pred = model.predict(x)
        
        y_true_mx = np.argmax(y_true)
        y_pred_mx = np.argmax(y_pred)
        
        equal += (y_true_mx == y_pred_mx)
        not_equal += (y_true_mx != y_pred_mx)
        
        if y_pred_mx == 0:
            continue
        
        output += REV_CLASSES_MAPPING[y_pred_mx]
    return output, equal, not_equal

In [ ]:
text = 'اللَّهُمَّ عَلِّمْنَا مَا يَنْفَعُنَا وَإِنْفَعْنَا بِمَا عَلَّمْتَنَا إِنَّكَ أَنْتَ العَلِيمُ الحَكِيمُ'
prediction = predict(text)
print(prediction[0], prediction[1], prediction[2], prediction[1] / (prediction[1] + prediction[2]) * 100, sep='\n')